In [1]:
# Imports

from modules import adquire, cleaning, create_df_checking, outputs


def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df = create_df_checking(df_students, df_katas)    
    outputs(df)


if __name__ == '__main__':
    main()

rsd13 presentará la kata user-class-for-banking-system


slug,tests-results,user-class-for-banking-system
username,,
4thSword,True,True
MaurizioMartin,True,True
farcDev0001,True,True
rsd13,True,True
Hermenegildo12,True,True
Nacho-Laiseca,True,True
rociobetegon,False,False
begoechavarren,True,True
inespchavarri,False,False
